In [37]:
from groq import Groq
import pandas as pd
import re
import os

# LLM API test

In [19]:
client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the average salary of a Engineer in Zurich Switzerland in CHF? only return the salary range in CHF",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

According to various sources, the average salary range for an engineer in Zurich, Switzerland is:

65,000 - 120,000 CHF per year


# Prediction loop for df

In [26]:
df_with_salary = pd.read_csv("../../data_acquisition/data/df_with_salary.csv")
df_without_salary = pd.read_csv("../../data_acquisition/data/df_without_salary.csv")
df_with_salary_few_shot = pd.read_csv("../../data_acquisition/data/df_with_salary_few_shot.csv")

In [27]:
df_with_salary_few_shot.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary
0,Consultant Microsoft 365 / Modern Workplace,Online Consulting AG,Festanstellung,Das wirst du machen Bereit für den ultimativen...,80,100,84000,102000
1,IT Supporter 80-100% (m/w/d),konekkt GmbH,Festanstellung,konekkt verbindet talentierte IT Fachkräfte mi...,100,100,65000,85000
2,Avaloq Developer (m/w/d) 80-100%,konekkt GmbH,Festanstellung,"Tritt einem führenden IT-Dienstleister bei, de...",80,100,105000,125000
3,Frontend Entwickler 80-100% (m/w/d),konekkt GmbH,Festanstellung,konekkt verbindet talentierte IT Fachkräfte mi...,100,100,95000,115000
4,Projektleiter,Ascaion AG,Festanstellung,Deine Aufgaben Unterstützung Pre-Sale durch A...,60,100,80000,140000


In [28]:
df_without_salary.head()

,title,company,contract_type,salary,description,min_workload,max_workload
0,Projektleiter / Consultant e-Case.Net,Infogate AG,Festanstellung,NaN,Wer braucht Deine Verstärkung? 1991 waren wir ...,80,100
1,Senior Software Engineer 80 - 100% (m/w/d),Wey Group AG,Festanstellung,NaN,Deine Aufgaben: Als Senior Software Engineer...,80,100
2,System Engineer - Ansible,RMIT Professional Resources AG,Freelance,NaN,System Engineer Position einhergehend mit Kenn...,80,100
3,"Softwareentwickler/in mit GIS-Erfahrung, Profe...",EBP Schweiz AG,Festanstellung,NaN,Dein Beitrag in unserem Team Du entwickelst b...,60,100
4,SAP Consultant PP-PI (m/w/d),Migros-Genossenschafts-Bund,Festanstellung,NaN,"""Teamwork makes the dream work"" ist für dich k...",60,100


In [ ]:
client = Groq(
    api_key="",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the average salary of a Engineer in Zurich Switzerland in CHF? only return the salary range in CHF",
        }
    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

In [ ]:
client = Groq(
    api_key="",
)
# Setup and load initial data
results_file = 'data/llm_predictions.csv'
if os.path.exists(results_file):
    df_without_salary = pd.read_csv(results_file)
else:
    df_without_salary['llm_min_salary'] = None  # Initialize columns if starting fresh
    df_without_salary['llm_max_salary'] = None
    df_without_salary['llm_answer'] = None 

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title} bei der Firma {company} liegt zwischen {min_salary} und {max_salary} mit einer Arbeitszeit von {min_workload} bis {max_workload} Prozent. Hier ist die Beschreibung, was Sie benötigen und tun müssen: {description}"
    }

def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})*) CHF für die Position {row['title']} bei der Firma {row['company']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def extract_salaries(response):
    pattern = re.compile(r"(\d{1,3}(?:['\s]\d{3})*)\s*(?:-|und)\s*(\d{1,3}(?:['\s]\d{3})*)\s*CHF")
    matches = pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "")) for salary in matches[0]]
        return min_salary, max_salary
    return None, None

def run_model(limit):
    start_index = df_without_salary['llm_min_salary'].last_valid_index() + 1 if df_without_salary['llm_min_salary'].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df_without_salary.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages=example_messages + [query_message],
            model="llama3-70b-8192",
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df_without_salary.at[index, 'llm_min_salary'] = min_salary
        df_without_salary.at[index, 'llm_max_salary'] = max_salary
        df_without_salary.at[index, 'llm_answer'] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df_without_salary.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
90000 120000
1
120000 150000
2
90000 120000
3
80000 120000
4
90000 120000
5
120000 180000
6
120000 150000
7
120000 150000
8
120000 150000
9
60000 80000
10
90000 140000
11
120000 180000
12
80000 120000
13
90000 120000
14
80000 120000
15
120000 150000
16
80000 120000
17
60000 90000
18
120000 180000
19
120000 180000
20
70000 90000
21
90000 120000
22
120000 160000
23
80000 120000
24
70000 110000
25
80000 120000
26
120000 150000
27
80000 110000
28
90000 120000
29
80000 120000
30
80000 110000
31
80000 110000
32
None None
33
120000 180000
34
80000 120000
35
45000 60000
36
110000 140000
37
80000 110000
38
None None
39
90000 120000
40
None None
41
80000 110000
42
80000 120000
43
80000 120000
44
80000 120000
45
90000 130000
46
120000 160000
47
80000 110000
48
70000 90000
49
70000 100000
50
90000 120000
51
90000 110000
52
90000 120000
53
None None
54
80000 120000
55
70000 90000
56
80000 110000
57
70000 100000
58
90000 120000


In [47]:
dflol = pd.read_csv("data/llm_predictions.csv")
dflol['llm_max_salary'].unique()

array([nan])